In [1]:
from collections import namedtuple
import scipy.optimize
import numpy as np

In [2]:
Food = namedtuple('Food', ['fat', 'carbs', 'protein', 'sodium', 'fiber', 'units'])

In [3]:
food = {
    'soymilk': Food(fat=1.99/243, carbs=10.01/243, protein=6/243, sodium=95/243, fiber=1/243, units='grams'),  # 1 cup = 243 g
    'egg': Food(fat=6.7, carbs=0.98, protein=6.09, sodium=88, fiber=0, units='large'),  # 1 large = 61 g
    'yogurt/maple syrup': Food(fat=0.01/170+0.0198, carbs=20.41/170, protein=18.01/170, sodium=67/170, fiber=0, units='grams yogurt'),  # 1 container = 170 g
    'yogurt': Food(fat=0.01/170, carbs=0, protein=18.01/170, sodium=67/170, fiber=0, units='grams'),  # 1 container = 170 g
    'maple syrup': Food(fat=0.0006, carbs=0.6704, protein=0.0004, sodium=0.12, fiber=0, units='grams'),  # 1 container = 170 g
    'blueberries': Food(fat=0.0033, carbs=0.1449, protein=0.0074, sodium=0.0001, fiber=0.0024, units='grams'),  # 1 container = 170 g
    'yogurt/blueberries/maple syrup': Food(fat=0.19/170, carbs=28.52/170, protein=18.42/170, sodium=68/170, fiber=1.3/170, units='grams yogurt'),  # 1 container = 170 g
    'celery': Food(fat=0.0017, carbs=0.0297, protein=0.0069, sodium=0.8, fiber=0.016, units='grams'),  # 100 g
    'peanut butter': Food(fat=9/16, carbs=3/16, protein=4/16, sodium=2.5/16, fiber=1/16, units='grams'),  # 2 tbsp = 32 g
    'carrot': Food(fat=0.0024, carbs=0.0958, protein=0.0093, sodium=0.69, fiber=0.028, units='grams'),  # 100 g
    'hummus': Food(fat=2.5/14, carbs=2/14, protein=1/14, sodium=62.5/14, fiber=0.5/14, units='grams'),  # 2 tbsp = 28 g
    'chicken': Food(fat=1.071, carbs=0, protein=9.306, sodium=22.2, fiber=0, units='ounces'),  # 100 g
    'broccoli': Food(fat=0.64/156, carbs=11.2/156, protein=3.72/156, sodium=64/156, fiber=5.2/156, units='grams'),  # 1/2 cup = 78 g
    'cheese': Food(fat=6, carbs=0, protein=7, sodium=190, fiber=0, units='sticks'),  # 1 stick
    'lettuce': Food(fat=0.0015, carbs=0.0287, protein=0.0136, sodium=0.28, fiber=0.013, units='grams'),  # 1 cup = 36 g
    'tomatoes': Food(fat=0.002, carbs=0.0389, protein=0.0088, sodium=0.05, fiber=0.012, units='grams'),  # 1 cup = 149 g
    'bell peppers': Food(fat=0.003, carbs=0.0603, protein=0.0099, sodium=0.04, fiber=0.021, units='grams'),  # 1 cup = 149 g
    'vinaigrette': Food(fat=(1+0)/2, carbs=(0+0.1703)/2, protein=(0+0.0049)/2, sodium=(0.02+0.23)/2, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'greek dressing': Food(fat=4.5/30, carbs=2/30, protein=0, sodium=280/30, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'protein powder': Food(fat=4/30, carbs=12/30, protein=11/30, sodium=0, fiber=12/30, units='grams'),
    'benefiber': Food(fat=0, carbs=7/7.4, protein=0, sodium=0, fiber=5/7.4, units='grams'),
    'almonds': Food(fat=0.4993, carbs=0.2155, protein=0.2115, sodium=0.01, fiber=0.125, units='grams'),
    'skirt steak': Food(fat=3.8, carbs=0, protein=9, sodium=0.189, fiber=0, units='ounce'),
    'salsa': Food(fat=0, carbs=2/30, protein=0, sodium=200/30, fiber=0, units='grams'),
    'cottage cheese': Food(fat=4.5, carbs=4, protein=13, sodium=400, fiber=0, units='cups'),
    'chicken parm': Food(fat=39, carbs=5.72, protein=70.5, sodium=358, fiber=1, units='serving'),
    'smoothie': Food(fat=0.92, carbs=51.22, protein=3.28, sodium=27, fiber=6.3, units='serving'),
    'tomato caper sauce': Food(fat=1.385, carbs=4.2075, protein=1.05, sodium=206.75, fiber=1.4, units='serving'),
    'salmon': Food(fat=1.8, carbs=0, protein=5.62, sodium=12, fiber=0, units='ounces'),
    'tilapia': Food(fat=0.79, carbs=0, protein=7.84, sodium=17, fiber=0, units='ounces'),
    'yogurt': Food(fat=0, carbs=9/227, protein=23/227, sodium=85/227, fiber=0, units='grams'),
    'fruit cup': Food(fat=0.09, carbs=25.13, protein=0.378, sodium=6.0, fiber=1.28, units='package and 20 g maple syrup'), # per 100 g drained, 1/2 cup = 77 g
    'chicken sausage': Food(fat=2, carbs=4, protein=5, sodium=160, fiber=0, units='link'),
    'pork sausage': Food(fat=0.248, carbs=0.0093, protein=0.1539, sodium=7.39, fiber=0, units='grams'), # 1 link = 25 g
    'almond milk': Food(fat=0.36, carbs=0.19, protein=0.19, sodium=23, fiber=0, units='ounces'),
    'banana': Food(fat=0.39, carbs=26.95, protein=1.29, sodium=1, fiber=3.1, units='medium'),
    'breaded chicken': Food(fat=1.68, carbs=5.81, protein=10.09, sodium=118, fiber=0.18, units='ounces'),
    'pork chop': Food(fat=1.24, carbs=0, protein=8.10, sodium=19, fiber=0, units='ounces'),
    'chicken meatballs': Food(fat=2.3, carbs=1.31, protein=4.18, sodium=117, fiber=0.48, units='meatball w/ sauce'),
    'cauliflower': Food(fat=0.0045, carbs=0.0411, protein=0.0184, sodium=0, fiber=0.023, units='grams'),
    'asparagus': Food(fat=0.0022, carbs=0.0411, protein=0.0240, sodium=0, fiber=0.020, units='grams'),
    'green beans': Food(fat=0.0028, carbs=0.0788, protein=0.0189, sodium=0, fiber=0.032, units='grams'),
    'canned green beans': Food(fat=0.0041, carbs=0.0419, protein=0.0105, sodium=0, fiber=0.019, units='grams'),
    'canned corn': Food(fat=0.0122, carbs=0.1434, protein=0.0229, sodium=0, fiber=0.020, units='grams'),
    'creamed corn': Food(fat=0.0042, carbs=0.1813, protein=0.0174, sodium=0, fiber=0.012, units='grams'),
    'gummies': Food(fat=0, carbs=3.5, protein=0, sodium=0, fiber=2, units='gummies'),
    'granola': Food(fat=6/30, carbs=18/30, protein=3/30, sodium=105/30, fiber=2/30, units='grams'),
    'muesli': Food(fat=3/30, carbs=19/30, protein=5/30, sodium=0/30, fiber=3/30, units='grams'),
}

In [4]:
target_cals = 2400
target_macros = {'fat': 75, 'carbs': 5, 'protein': 20, 'fiber': 20}  # fat loss
#target_macros = {'fat': 15, 'carbs': 55, 'protein': 30, 'fiber': 30}  # muscle gain
targets = {'fat': target_macros['fat'] / 9 * target_cals / 100,
           'carbs': target_macros['carbs'] / 4 * target_cals / 100,
           'protein': target_macros['protein'] / 4 * target_cals / 100,
           'fiber': target_macros['fiber']}
print('targets =', targets)
amounts = {
#    'yogurt/maple syrup': 170,
    'yogurt': (0,280),
    'blueberries': (0,36),
    'muesli': (0,23),
    'maple syrup': (0,14),
    'celery': (0,65),
    'peanut butter': (0*32,1.2*32),
    'carrot': (0,65),
    'hummus': (0*28,1.2*28),
    'chicken': (0,16),
    'smoothie': (0,1),
    'protein powder': (0,30),
    'broccoli': (0,117),
    'cheese': (0,1),
    'lettuce': (0,108),
    'tomatoes': (0,75),
    'greek dressing': (0,30),
    'gummies': (0,4),
    'cottage cheese': (0,1),
  #  'fruit cup': 1,
}
ints = ['egg', 'cheese']

targets = {'fat': 200.0, 'carbs': 30.0, 'protein': 120.0, 'fiber': 20}


In [5]:
calc_targets = targets.copy()
for name, ran in amounts.items():
    if not isinstance(ran, tuple):
        calc_targets['fat'] -= ran * food[name].fat
        calc_targets['carbs'] -= ran * food[name].carbs
        calc_targets['protein'] -= ran * food[name].protein
        calc_targets['fiber'] -= ran * food[name].fiber
calc_target_cals = int(calc_targets['fat'] * 9 + calc_targets['carbs'] * 4 + calc_targets['protein'] * 4)
calc_food = [(name, amt) for name, amt in amounts.items() if isinstance(amt, tuple)]
names = [entry[0] for entry in calc_food]

def prn():
    y = np.zeros(5)
    for nam, val in amounts.items():
        fd = food[nam]
        y += val * np.array([fd.fat, fd.carbs, fd.protein, fd.fiber, fd.sodium])
        print(f'    {nam}: {val} {fd.units}')
    cals = y[0]*9 + y[1]*4 + y[2]*4
    print(f'Total Calories = {int(cals)}, Fat = {y[0]:4.1f}, Carbs = {y[1]:.1f}, Protein = {y[2]:5.1f}, Sodium = {int(y[4]):4d}, Fiber = {y[3]:4.1f}')
    print(f'   Target Cals = {target_cals}, Fat = {targets["fat"]:.1f}, Carbs = {targets["carbs"]:.1f}, Protein = {targets["protein"]:.1f}, Sodium < 2000, Fiber = {targets["fiber"]:.1f}')
    print(f'Total Macros:  Fat = {y[0]*9/cals*100:4.1f}%, Carbs = {y[1]*4/cals*100:4.1f}%, Protein = {y[2]*4/cals*100:4.1f}%')
    print(f'     Targets:  Fat = {target_macros["fat"]:4.1f}%, Carbs = {target_macros["carbs"]:4.1f}%, Protein = {target_macros["protein"]:4.1f}%')

def foo(x):
    y = np.zeros(5)
    for i, val in enumerate(x):
        fd = food[names[i]]
        y += val * np.array([fd.fat, fd.carbs, fd.protein, fd.fiber, fd.sodium])
    arr = np.array([calc_targets['fat'], calc_targets['carbs'], calc_targets['protein'], calc_targets['fiber']]) - y[0:4]
    arr[3] = max(arr[3], 0.0)
    return np.linalg.norm(arr)

def run(algorithm):
    cons = lambda x: max(x[i] - int(x[i]) if names[i] in ints else 0 for i in range(len(x)))
    if algorithm == 'basinhopping':
        x0 = np.array([a[1][0] for a in calc_food])
        res = scipy.optimize.basinhopping(
            foo, x0, niter=400,
            minimizer_kwargs={'bounds': [a[1] for a in calc_food]})
        res = res.x
    if algorithm == 'brute':
        bounds = [a[1] for a in calc_food]
        res = scipy.optimize.brute(foo, bounds, finish=None)
    res = np.rint(res).astype(int)
    for nam, amt in zip(names, res):
        amounts[nam] = amt
    prn()
    for nam, amt in amounts.items():
        maple = int(np.rint(amt*20/170))
        blueberries = int(np.rint(amt*56/170))
        if nam == 'yogurt/maple syrup':
            print(f'{maple} grams of maple syrup')
        elif nam == 'yogurt/blueberries/maple syrup':
            blueberries = amt*56/170
            print(f'{maple} grams of maple syrup and {blueberries} grams of blueberries')

In [6]:
run('basinhopping')

    yogurt: 0 grams
    blueberries: 0 grams
    muesli: 23 grams
    maple syrup: 0 grams
    celery: 0 grams
    peanut butter: 38 grams
    carrot: 0 grams
    hummus: 34 grams
    chicken: 10 ounces
    smoothie: 0 serving
    protein powder: 30 grams
    broccoli: 0 grams
    cheese: 1 sticks
    lettuce: 0 grams
    tomatoes: 0 grams
    greek dressing: 30 grams
    gummies: 0 gummies
    cottage cheese: 1 cups
Total Calories = 1272, Fat = 59.5, Carbs = 44.5, Protein = 139.8, Sodium = 1249, Fiber = 17.9
   Target Cals = 2400, Fat = 200.0, Carbs = 30.0, Protein = 120.0, Sodium < 2000, Fiber = 20.0
Total Macros:  Fat = 42.0%, Carbs = 14.0%, Protein = 43.9%
     Targets:  Fat = 75.0%, Carbs =  5.0%, Protein = 20.0%


In [7]:
run('brute')

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [ ]:
    yogurt/maple syrup: 227 grams yogurt
    celery: 65 grams
    peanut butter: 38 grams
    carrot: 65 grams
    hummus: 11 grams
    chicken: 13 ounces
    broccoli: 78 grams
    cheese: 1 sticks
    lettuce: 72 grams
    tomatoes: 75 grams
    greek dressing: 30 grams
    benefiber: 19 grams
    cottage cheese: 1 cups
    fruit cup: 1 package and 20 g maple syrup
Total Calories = 1615, Fat = 53.2, Carbs = 103.8, Protein = 180.2, Sodium = 1461, Fiber = 24.2
   Target Cals = 1600, Fat = 53.3, Carbs = 100.0, Protein = 180.0, Sodium < 2000, Fiber = 30.0
Total Macros:  Fat = 29.7%, Carbs = 25.7%, Protein = 44.6%
     Targets:  Fat = 30.0%, Carbs = 25.0%, Protein = 45.0%
27 grams of maple syrup